### Fed H.4.1

Fed Data on Factors Affecting Reserve Balances

Thursday 4:30PM

In [1]:
import sys
sys.path.append('../src')

import time

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
# Retrieve data from Fed DDP
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=H41&series=d898b3af70d8e05b2f639b3f06b0f93f&lastobs=&'+
       'from=01/01/2002&to=12/31/2022&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)

# Fed Balance Sheet Total
bst = (df['RESPPMA_N.WW'] / 1_000_000)
bst.to_csv(data_dir / 'fed_assets.csv', index_label='date', 
           float_format='%g')

# Fed Balance Sheet Text
ltval = bst.iloc[-1].round(1)
ltdate = dtxt(bst.index[-1])['day1']
cl = c_line('blue!80!black')
text = ('In response to the collapse of the housing bubble, the '+
        'Fed purchased U.S. Treasury bonds and mortgage-backed '+
        f'securities. Total assets held by the Federal Reserve {cl} '+
        'increased from \$0.9 trillion in August 2008 to \$2.2 '+
        'trillion in November 2008. Additional rounds of asset '+
        'purchases, referred to as quantitative easing, increased '+
        'the balance sheet to \$4.5 trillion by January 2014. As '+
        'bonds mature they were replaced until October 2017, when '+
        'the Fed allowed the size of its balance sheet to normalize. '+
        'Total assets fell below \$3.8 trillion in August 2019.')
write_txt(text_dir / 'fed_assets1.txt', text)
        
txt2 = ('Balance sheet normalization ended in September 2019 when '+
        'the Fed increased operations in overnight and term repurchase '+
        'agreement (repo) markets, following a sharp increase in rates '+
        'in these markets. The Fed balance sheet increased to \$4.1 '+
        'trillion by December 2019.')
write_txt(text_dir / 'fed_assets2.txt', txt2)
        
txt3 = ('During the COVID-19 pandemic, the Fed offered lending to '+
        'businesses and currency swaps to major US trading partners, '+
        'began to purchase commercial bonds, and expanded purchases of '+
        'treasuries and mortgage-backed securities.\n\n The Fed balance '+
        'sheet increased from \$4.2 trillion in February 2020 to '+
        f'\${ltval} trillion, as of the latest data, covering {ltdate}. ')
write_txt(text_dir / 'fed_assets3.txt', txt3)

# Fed Balance Sheet Components Table
d2 = {'RESPPA_N.WW': f'\\textbf{{Total}} {cl}',
      'RESPPALGUO_N.WW': '\hspace{2mm}U.S. Treasury securities',
      'RESPPALGASMO_N.WW': '\hspace{2mm}Mortgage-backed securities',
      'RESH4SCS_N.WW': '\hspace{2mm}Central bank liquidity swaps',
      'RESPPALGTR_N.WW': '\hspace{2mm}Repurchase agreements',
      'RESPPALD_N.WW': '\hspace{2mm}Loans',
      'RESPPALDJ_N.WW': '\hspace{4mm}Payroll Protection Program',
      'NetPremDisc': '\hspace{2mm}Net unamortized premium',
      'Other': '\hspace{2mm}Other'}

pr_di = ['RESPPALSD_N.WW', 'RESPPALSP_N.WW']

data = df.copy()
dropcol = ['RESPPA_N.WW', 'RESPPLLOP_N.WW', 'RESPPMA_N.WW']
data['Other'] = (data['RESPPA_N.WW'] - data.drop(dropcol,
                                                 axis=1).sum(axis=1))
data['NetPremDisc'] = data[pr_di].sum(axis=1)
ltval_treas = data['RESPPALGUO_N.WW'].iloc[-1] / 1_000_000
ltval_mbs = data['RESPPALGASMO_N.WW'].iloc[-1] / 1_000_000

data = data.rename(d2, axis=1).drop(pr_di, axis=1)
dft = pd.DataFrame()
for i in [-1, -2, -5, -53, -105]:
    dft[dtxt(data.index[i])['day2']] = data.iloc[i]
    
(dft.loc[list(d2.values()), :].div(1000)
    .applymap('{:,.1f}'.format)
    .to_csv(data_dir / 'fed_bal_sheet.tex', sep='&', 
            line_terminator='\\\ ', quotechar=' '))

text = (f'The Fed currently holds \${ltval_treas:,.1f} trillion in '+
        f'Treasuries and \${ltval_mbs:,.1f} trillion in mortgage-'+
        'backed securities. ')
write_txt(text_dir / 'fed_assets4.txt', text)
print(text)

# Adjustment for Fed Interest Expense
res = (df['RESPPLLOP_N.WW'].rolling(52).sum()
         .resample('MS').mean().dropna())
fedadj = res.loc[res.index.month == 10]
fedadj.to_csv(data_dir / 'fedintadj.csv', index_label='date')

The Fed currently holds \$5.6 trillion in Treasuries and \$2.7 trillion in mortgage-backed securities. 
